In [1]:
import numpy as np
import pandas as pd

from mh_utils import (
    AdaptiveImportanceSampling, PermutationPropsal, PermutationStatistic, MCMCISPermutationTest
)
from samc import SAMC
from mcmcis import SmoothedMCMCIS

import seaborn as sns

import numpy as np
from itertools import combinations
from math import comb


In [2]:
def permutation_test(
    s1,
    s2,
    stat=None,
    alternative="greater",   # "greater" | "less" | "two-sided"
    exact=True,
    max_enumerations=5_000_000,
):
    """
    Exact (or MC) permutation test for two samples.

    Parameters
    ----------
    s1, s2 : array-like
        Samples (1-D).
    stat : callable or None
        Test statistic function stat(s1, s2) -> float.
        If None: uses difference of means: mean(s1) - mean(s2).
    alternative : {"greater","less","two-sided"}
        Tail definition relative to the observed statistic.
    exact : bool
        If True, enumerate all label permutations (over group membership).
        If the number of permutations exceeds `max_enumerations`, falls back
        to Monte Carlo with `n_resamples`.
    max_enumerations : int
        Hard cap on number of exact permutations to iterate.
    n_resamples : int
        Number of Monte Carlo permutations if exact is infeasible.
    random_state : int or None
        Seed for Monte Carlo.

    Returns
    -------
    p_value : float
        Estimated (exact or MC) p-value.
    observed : float
        Observed statistic on the original split.
    enumerations_used : int
        Number of permutations actually evaluated.
    exact_flag : bool
        True if exact enumeration was used; False if Monte Carlo fallback.
    """
    s1 = np.asarray(s1, dtype=float).ravel()
    s2 = np.asarray(s2, dtype=float).ravel()
    n1, n2 = len(s1), len(s2)
    N = n1 + n2
    pooled = np.concatenate([s1, s2])

    if stat is None:
        def stat(a, b):  # difference in means
            return float(np.mean(a) - np.mean(b))

    observed = stat(s1, s2)


    total_perms = comb(N, n1)

    # If exact is requested but huge, fallback to MC.
    if exact and total_perms <= max_enumerations:
        count = 0
        # Enumerate all combinations of indices for group 1
        # Generic: build groups explicitly
        all_idx = np.arange(N)
        for idx in combinations(range(N), n1):
            mask = np.zeros(N, dtype=bool)
            mask[list(idx)] = True
            t = stat(pooled[mask], pooled[~mask])
            if _tail_indicator(t, observed, alternative):
                count += 1

        # p-value with standard permutation “>=” counting and no +1 (exact)
        p_value = count / total_perms
        return float(p_value), float(observed), int(total_perms), True

    else: raise ValueError
    
def _tail_indicator(t, t_obs, alternative):
    """Return True if t is at least as extreme as t_obs in the requested tail."""
    if alternative == "greater":
        return t >= t_obs
    if alternative == "less":
        return t <= t_obs
    if alternative == "two-sided":
        # Two-sided by absolute deviation from the null center (0 for diff of means).
        # If you use a general stat not centered at 0 under H0, adapt this.
        return abs(t) >= abs(t_obs)
    raise ValueError("alternative must be 'greater', 'less', or 'two-sided'")

In [3]:
class DifferenceInMeans(PermutationStatistic):
    def __call__(self, s1, s2):
        return np.mean(s1) - np.mean(s2)




class SwapProposal(PermutationPropsal):
    def __init__(self, swap_size: int):
        self.swap_size = swap_size
    
    def __call__(self, s1, s2):
        # choose random indices
        idx_s1 = np.random.choice(s1.shape[0], self.swap_size, replace=False)
        idx_s2 = np.random.choice(s2.shape[0], self.swap_size, replace=False)

        new_s1, new_s2 = s1.copy(), s2.copy()
        new_s1[idx_s1] = s2[idx_s2]
        new_s2[idx_s2] = s1[idx_s1]

        return new_s1, new_s2


In [4]:
np.random.seed(111)

s1 = np.random.poisson(5, 15)
s2 = np.random.poisson(2, 10)


diff_in_means = DifferenceInMeans()
proposal_swaps = SwapProposal(swap_size=2)

samc = SAMC(
    lambda_star=diff_in_means(s1, s2),
    num_regions=100,
    lower_bound = -1,
    t0=10_000
)

In [5]:
smcmcis = SmoothedMCMCIS(
    s1=s1,
    s2=s2,
    lambda_star=diff_in_means(s1, s2),
    target_prob=0.01,
    statistic_func=diff_in_means
)

In [6]:
smoothed_mcmcis = MCMCISPermutationTest(
    s1=s1,
    s2=s2,
    J=4,
    T=1_000_00,
    B=25_000,
    ais=smcmcis,
    statistic_func=diff_in_means,
    proposal_func=proposal_swaps,
    variance_estimation_method="whatever",
    log_scale=False
)

smoothed_mcmcis.run_chain()

### Starting Adaptation Chain 1/4 ###
p-value = 0.0004568424543735417
Acceptance rate = 0.7718
### Starting Adaptation Chain 2/4 ###
p-value = 0.0003502502093686491
Acceptance rate = 0.671
### Starting Adaptation Chain 3/4 ###
p-value = 0.00039970677304580423
Acceptance rate = 0.6373
### Starting Adaptation Chain 4/4 ###
p-value = 0.0003911984719842857
Acceptance rate = 0.6201


In [ ]:
samc_mcmcis = MCMCISPermutationTest(
    s1=s1,
    s2=s2,
    J=2,
    T=1_000_000,
    B=25_000,
    ais=samc,
    statistic_func=diff_in_means,
    proposal_func=proposal_swaps,
    variance_estimation_method="whatever",
    log_scale=True
)

samc_mcmcis.run_chain()

### Starting Adaptation Chain 1/2 ###


In [ ]:
# Simple example
p, t_obs, K, is_exact = permutation_test(s1, s2, alternative="greater", exact=True)
print(p)

0.0003876087568374552
